# Pilotage d'une LED RGB _- 40 ans de télé couleur -_

Aujourd'hui une partie du marché des téléviseurs est occupé par des appareils à LED (diode électroluminescente). Dans le monde des objets connectés on retrouve des écrans de petite taille (64 points) utilisant la même technologie. Avant de passer à de tels instruments complexes nous allons voir comment produire toutes les couleurs de l'arc en ciel avec une diode RVB _Rouge-Vert-Bleu_ ou RGB _Red-Green-Blue_; le noir étant visible quand la diode est éteinte et le blanc .... étant une lumière vive.

La LED RGB comprend trois diodes aux propriétés différentes qui produisent une couleur différente, des résistances permettent de modifier l'intensité du courant qui les parcourt et ajuster l'éclairement entre elles pour obtenir le mélange de couleurs souhaité et afficher la bonne couleur. Dans ce qui suit nous utilisons une LED RGB à **cathode commune** ce qui signifie que nous avons une broche commune reliée au _-_ et une broche par couleur reliée à la source de courant. Nous proposons un branchement direct utilisant une source 3.3V dans la figure ci-dessous. 

![Montage d'une LED RGB à cathode commune](../images/ledboard5.png)
                        ![licence : Gérard Vidal](../images/Licence.jpg)
                        
Les trois connecteurs permettent de relier chacune des résistances liée à l'une des broches de la diode on utilise les ports GPIO numéro 22, 23 et 24qui correspondent aux broches 15 16 et 18 de la plaque de raccordement T-Cobbler. Les broches sont réparties selon l'ordre des couleurs du spectre R puis V puis B.

Nous allons d'abord nous familiariser avec cette LED RGB avec une seule couleur de base puis produire des couleurs à la demande et éventuellement réaliser un Arc en ciel.

##  Comprendre le fonctionnement des leds RGB

Quelques lignes de code pour tester trois LEDs Rouge vert bleu contenues dans la LED RGB. Pour contrôler l'intensité lumineuse d'une LED on ne fait pas varier l'intensité du courant qui la traverse ! On utilise un disposif de contrôle appelé modulation de la largeur d'impulsion _PWM_ pour Pulse Width Modulation. 

En quelques mots... Le courant qu'on envoie dans la LED ne peut être que "à fond" ou "pas du tout", l'intensité est soit maximale soit nulle. Cela ne semble donc pas facile de faire varier l'intensité d'une LED  avec cette contrainte ! pourtant.... On tire profit des propriétés de l'oeil humain qui ne parvient pas à déceler convenablements des événements de très courte durée et se contente de fournir une mesure moyenne pendant "un certain temps" (raison pour laquelle 24 ou 25 images par seconde donnent l'illusion d'un mouvement fluide au cinéma ou à la télévision[nous y voilà !]). Notre Raspberry Pi est capable lui au contraire d'allumer ou d'éteindre la LED pendant une durée très très courte au point que l'oeil "vous fera croire" que la LED est éteinte alors qu'elle s'allume pendant une série de durées extrêmement courtes ou qu'elle est allumée alors qu'elle s'éteint pendant une série de durées extrêmement courtes. L'astuce est donc de choisir une durée suffisamment courte pour que l'oeil ne la détecte pas (disons 20 millisecondes) et d'allumer la LED pendant la moitié de ce temps (soit 10 millisecondes) puis de répéter ce cycle _allumé-éteint_ 50 fois (soit une seconde) l'oeil va faire la moyenne et vous fera croire que la diode a été "à moitié" éclairée pendant une seconde. Ainsi en ayant éclairé la diode à pleine puissance **la moitié du temps** pendant une série de durées imperceptibles pour l'oeil vous aurez eu l'impression que la diode était éclairée avec **la moitié de son intensité**.

Le but de ce petit code est de vérifier que les broches activées en PWM sont effectivement opérationnelles et tester la valeur des résistances à mettre en oeuvre pour l'éclairement soit homogène entre les différentes couleurs. 

On utilise la librairie RPi.GPIO qui contient des codes permettant de mettre en oeuvre du PWM sur des broches GPIO normales (On peut aussi configurer le raspberry pour certaines broches soient en mode PWM plus performant mais cette manipulation demande des compétences plus avancées et nous pouvons nous en passer grace à RPi.GPIO.

On inclut la librairie et on oriente les broches.

In [1]:
import RPi.GPIO as gpio, time
gpio.setmode(gpio.BOARD)  # BCM ou BOARD pour identifier les broches. choisir BOARD

dur = 0.25
gpio.setup(15, gpio.OUT)# GPIO 22 broche 15 output  
gpio.setup(16, gpio.OUT)# GPIO 23 broche 16 output 
gpio.setup(18, gpio.OUT)# GPIO 24 broche 18 output  

On définit le fonctionnement PWM sur les broches considérées. Le temps "imperceptible" pour l'oeil choisi ici est 20 millisecondes  qui nous permettrond de répéter l'opération _allumé-éteint_ 50 fois par seconde  soit 50 Hertz

In [2]:
pb = gpio.PWM(15, 50)    # led bleue crée un objet pg  PWM sur le port GPIO22 à 50 Hertz  T=20ms 
pg = gpio.PWM(16, 50)    # led verte crée un objet pb  PWM sur le port GPIO23 à 50 Hertz  T=20ms 
pr = gpio.PWM(18, 50)    # led rouge crée un objet pr  PWM sur le port GPIO24 à 50 Hertz  T=20ms 

load = 10

pr.start(load)             # allumage à load % (load% duty cycle)   
pb.start(load)             # allumage à load % (load% duty cycle)   
pg.start(load)             # allumage à load % (load% duty cycle)

On arrête et on nettoie

In [3]:
pr.stop()
pg.stop()
pb.stop()
gpio.cleanup()          #nettoyage de l'état du gpi

Pas à pas sur la LED Rouge

In [4]:
import RPi.GPIO as gpio, time
gpio.setmode(gpio.BOARD)  # BCM ou BOARD pour identifier les broches. choisir BOARD

dur = 0.25
gpio.setup(18, gpio.OUT)# GPIO 24 broche 18 output  
pr = gpio.PWM(18, 50)    # led rouge crée un objet pr  PWM sur le port GPIO24 à 50 Hertz  T=20ms 

On choisit le pourcentage de temps pendant lequel la diode sera allumée : la charge (load)

In [5]:
load = 20

On démarre la LED Rouge

In [6]:
pr.start(load)             # allumage à load % (load% duty cycle) 

On éteint la LED Rouge

In [7]:
pr.stop()

On nettoie le bus quand tout est fini. L'instruction suivante efface toutes les configurations des broches concernées'

In [8]:
gpio.cleanup()          #nettoyage de l'état du gpio

### Exercice 1

Écrivez un programme ci dessous pour :
 * allumer la diode rouge pendant 0.25s à 15% de la puissance max (fréquence 200Hz)
 * augmenter la puissance jusqu'à 80% par pas de 5% et intervalle de 0.25s
 * faire clignoter la LED à une fréquence de 8Hz Rouge 15 Hz Vert et 18 Hz bleu
 
Faites de même pour la LED verte et la LED bleue avec l'intensité et la fréquence de votre choix.
 

### Exercice 2

Faire de même avec les couleurs complémentaires :
Jaune Magenta Cyan

Un indice :
Jaune = Rouge + Vert
Magenta = Rouge + Bleu
Cyan = Bleu + Vert

Il est souhaitable de réfléchir avec un papier et un crayon avant de taper sur les touches ;-)

# Modulation d'une LED RGB : création d'un arc-en-ciel

Ce chapitre aborde la combinaison de la lumière Rouge verte et bleue issue d'une LED RGB pour produire toutes les couleurs du spectre. L'objectif est de parvenir à reproduire l'enchainement des couleurs avec une LED RGB comme présenté sur ce [petit film](http://mediaserv.climatetmeteo.fr/users/GerardVidal/ObjetsConnectes/lePotDeYaourtMagique.mp4).

 Pour pouvoir lancer une boucle infinie et détecter les entrées au clavier pendant l'excution des tâches une nouvelle bibliothèque est ajoutée ainsi que le contrôle des commandes émises par le clavier.

Déclarations pour le programme

In [6]:
import RPi.GPIO as gpio, time
gpio.setmode(gpio.BOARD)  # choisir BCM ou BOARD pour identifier les broches. Ici BOARD
import _thread as thread  # permet d'exécuter le programme tout en permettant de détecter les touches enfoncées

gpio.setup(15, gpio.OUT)# GPIO 22 broche 15 output  
gpio.setup(16, gpio.OUT)# GPIO 23 broche 16 output 
gpio.setup(18, gpio.OUT)# GPIO 24 broche 18 output  

pg = gpio.PWM(15, 200)    # led verte crée un objet pg  PWM sur le port GPIO22 à 200 Hertz  T=5ms 
pb = gpio.PWM(16, 200)    # led bleue crée un objet pb  PWM sur le port GPIO23 à 200 Hertz  T=5ms 
pr = gpio.PWM(18, 200)    # led rouge crée un objet pr  PWM sur le port GPIO24 à 200 Hertz  T=5ms 

À titre d'exemple programme permettant d'augmenter puis de réduire la luminosité rouge de la LED.

In [7]:
dur = 0.15

# veille sur l'input
def input_thread(CheckInput):
    check = input()
#    CheckInput = CheckInput.append(check)
    CheckInput.append(check)
    return

def rainbow():
    CheckInput = []
    thread.start_new_thread(input_thread, (CheckInput,))
    print ('Type "stop" to stop modulation')
            
    while True :
        pr.start(0)             # allumage à 0 % (0% duty cycle) 
        time.sleep(dur)
        for i in range(0,100,2) :
            pr.ChangeDutyCycle(i)   # de 0 à 100% par pas de 2% par 0.25s 
            time.sleep(dur)
        time.sleep(dur)
        for i in range(100,0,-2) :
            pr.ChangeDutyCycle(i)   # de 0 à 100% par pas de 2% par 0.25s
            time.sleep(dur)
        time.sleep(dur)
        
        # Sortie de boucle infinie
        if CheckInput == ['stop']  : 
            pr.stop()
            break
    return


rainbow()


Type "stop" to stop modulation
stop


In [8]:
pr.stop()

In [9]:
gpio.cleanup()          #nettoyage de l'état du gpio  

### Exercice 3 

Réalisez un programme permettant de parcourir toutes les couleurs de l'arc en ciel.